# Data Exploration

author = Caroline Magg <br>
date = 31 March 2020 <br>

___________________________________
history: <br>
2020-03-31 <br>
inspect single folder content <br>
write methods for reading structure, contour names and contour content <br>

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import pydicom
import vtk

In [ ]:
from utils_read import read_structure,read_contour,read_contour_names,read_contour_row

In [ ]:
%matplotlib inline

### Add paths and dependencies
This can vary depending on your environment

In [ ]:
# add KidsBrainProject main folder to paths
sys.path.append(os.path.abspath('../../'))

In [ ]:
# add path to data here
path_data = "../../Data/" 

# Inspect single folder content

In [ ]:
idx = 1
folder_name = os.path.join(path_data, str(idx))
folder = os.listdir(folder_name)
folder

### CT

In [ ]:
folder_ct_name = os.path.join(path_data, str(idx),'CT')
folder_ct = os.listdir(folder_ct_name)
folder_ct[:5]

In [ ]:
df_ct = read_structure(folder_ct_name)
len(df_ct)

### MRI

In [ ]:
folder_mri_name = os.path.join(path_data, str(idx),'T1 +C 3-15-16')
folder_mri = os.listdir(folder_mri_name)
folder_mri[:5]

In [ ]:
df_mri = read_structure(folder_mri_name)
len(df_mri)

### Contours

In [ ]:
path_contours = os.path.join(path_data, str(idx), 'RS.Jacks1.dcm')

In [ ]:
df_contours = read_contour_names(path_contours)
len(df_contours)

In [ ]:
contours = read_contour(path_contours, df_contours)
len(contours)